# Imports

In [1]:
import pandas as pd
import numpy as np

from mysql.connector import MySQLConnection, Error
from configparser import ConfigParser

from db_helpers import read_db_config, execute, insert

import json

# Batting perf table

In [ ]:
table_name = "batting_perf"

q = f"""
drop table if exists {table_name};
"""

execute(q)



q = f"""
create table {table_name} (
match_num int NOT NULL,
team varchar(30) not null,

toss varchar(20),
toss_result varchar(20),

overall_RR DECIMAL(6,4),
avg_RR DECIMAL(6,4),
avg_req_RR DECIMAL(6,4),

num_6s int,
num_4s int,

match_result varchar(20) not null

);
"""

execute(q)

In [ ]:
dbconfig = read_db_config()
conn = MySQLConnection(**dbconfig)

cursor = conn.cursor(dictionary=True)

cols = "(match_num, team, toss, toss_result, overall_RR, avg_RR, avg_req_RR, num_6s, num_4s,match_result)"
tmp = "values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) "
vals = []

query = f"insert into {table_name} {cols} "+tmp


q = """

SELECT 
b.match_num,
b.team,
b.toss,
b.toss_result,

-- overall RR
round(b.score_runs/(b.score_balls_faced/6),4) AS overall_RR,

-- avg RR
CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[0].inningOvers[*].overRunRate" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[1].inningOvers[*].overRunRate" END AS RR,

-- avg req RR
CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[0].inningOvers[*].requiredRunRate" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[1].inningOvers[*].requiredRunRate" END AS req_RR,


-- ball wise 6s
CASE WHEN b.toss_result = "BAT" THEN o.data ->> "$.inningOvers[0].stats[*].balls[*].isSix" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[1].stats[*].balls[*].isSix" END AS num_6s,

-- ball wise 4s
CASE WHEN b.toss_result = "BAT" THEN o.data ->> "$.inningOvers[0].stats[*].balls[*].isFour" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[1].stats[*].balls[*].isFour" END AS num_4s,


b.match_result

FROM scoreboard b
LEFT JOIN scorecard c ON b.match_num = c.match_num
LEFT JOIN overs o ON b.match_num = o.match_num
ORDER BY b.match_num;

"""


cursor.execute(q)

row = cursor.fetchone()

vals = []

while row is not None:
    try:
        tmp = (
            row['match_num'],
            row['team'],
            row['toss'],
            row['toss_result'],
            row['overall_RR'],
            float(np.round(np.average(json.loads(row['RR'])) ,4)),
            float(np.round(np.average(json.loads(row['req_RR'])) ,4)) if np.average(json.loads(row['req_RR'])) != 0 else None ,
            int(np.sum(json.loads(row['num_6s']))),
            int(np.sum(json.loads(row['num_4s']))),
            row['match_result']
        )
    except:
            tmp = (
            row['match_num'],
            row['team'],
            row['toss'],
            row['toss_result'],
            row['overall_RR'],
            None,
            None,
            None,
            None,
            row['match_result']
        )


    print(tmp)

    vals.append(tmp)
    
    row = cursor.fetchone()
    


insert(query,vals,commit=True)

# dist_batting_perf table

In [18]:
def make_len_constant(arr,length=50, replace_val=None, dtype=float):
    return np.pad(np.array(arr, dtype=dtype), (0, length-len(arr)),'constant', constant_values=replace_val)


def reduce_len(arr, factor,normalise=False,func='mean', nan_replace_val=None):
    
    if func == 'mean':
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).mean(axis=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).mean(axis=1).replace(np.nan,None).to_list()
 
    if func == 'median':
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).median(axis=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).median(axis=1).replace(np.nan,None).to_list()

    if func == 'sum':
        #         pandas fix issue of if all na, it returns 0.0, to fix do min_count=1
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).sum(axis=1,min_count=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).sum(axis=1,min_count=1).replace(np.nan,None).to_list()
    
    
    


table_name = "dist_batting_perf"

q = f"""
drop table if exists {table_name};
"""

execute(q)


q = f"""
create table {table_name} (
match_num int NOT NULL,
team varchar(30) not null,

toss varchar(20),
toss_result varchar(20),

overs varchar(8),
avg_RR DECIMAL(6,4),
avg_req_RR DECIMAL(6,4),

num_6s int,
num_4s int,
num_wickets int,
dist_num_6s DECIMAL(6,4),
dist_num_4s DECIMAL(6,4),
dist_num_wickets DECIMAL(6,4),

match_result varchar(20) not null

);
"""

execute(q)

dbconfig = read_db_config()
conn = MySQLConnection(**dbconfig)

cursor = conn.cursor(dictionary=True)

cols = "(match_num, team, toss, toss_result, overs, avg_RR, avg_req_RR, num_6s, num_4s, num_wickets, dist_num_6s, dist_num_4s, dist_num_wickets, match_result)"
tmp = "values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) "
vals = []

query = f"insert into {table_name} {cols} "+tmp


q = """

SELECT 
b.match_num,
b.team,
b.toss,
b.toss_result,

-- runs -> will get sectorwise RR
CASE WHEN b.toss_result = "BAT" THEN o.data ->> "$.inningOvers[0].stats[*].overRuns" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[1].stats[*].overRuns" END AS runs,

-- req_RR -> can get sectorwise avg(req_RR)
CASE WHEN b.toss_result = "BAT" THEN o.data ->> "$.inningOvers[0].stats[*].requiredRunRate" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[1].stats[*].requiredRunRate" END AS req_RR,

CASE WHEN b.toss_result = "BAT" THEN o.data -> "$.inningOvers[0].stats[*]" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[1].stats[*]" END AS overs_data,

b.match_result

FROM scoreboard b
LEFT JOIN scorecard c ON b.match_num = c.match_num
LEFT JOIN overs o ON b.match_num = o.match_num
ORDER BY b.match_num;

"""


cursor.execute(q)

row = cursor.fetchone()

vals = []
num = 1
while row is not None:
    num_6s_overs = []
    num_4s_overs = []
    num_wickets_overs = []

    tmp_over_6s = []
    tmp_over_4s = []
    tmp_wickets_overs = []

    try:
        for i in json.loads(row['overs_data']):
            for j in i['balls']:
                tmp_over_6s.append(j['isSix'])
                tmp_over_4s.append(j['isFour'])
                tmp_wickets_overs.append(j['isWicket'])

            num_6s_overs.append(np.sum(np.array(tmp_over_6s,dtype=int)))
            num_4s_overs.append(np.sum(np.array(tmp_over_4s,dtype=int)))
            num_wickets_overs.append(np.sum(np.array(tmp_wickets_overs,dtype=int)))

            tmp_over_6s = []
            tmp_over_4s = []
            tmp_wickets_overs = []
    except:
        num_6s_overs = np.full((50),None).tolist()
        num_4s_overs = np.full((50),None).tolist()
        num_wickets_overs = np.full((50),None).tolist()


    overs = ['0-5','5-10','10-15','15-20','20-25','25-30','30-35','35-40','40-45','45-50']


    # avgRR
    try:
        avg_RR = reduce_len(
            arr = make_len_constant(json.loads(row['runs'])),
            factor = 5,
            func='mean',
        )
    except:
        avg_RR = np.full((10),None).tolist()


    # avg_req_RR
    try:
        avg_req_RR = reduce_len(
            arr = make_len_constant(json.loads(row['req_RR'])),
            factor = 5,
            func='median'
        )
    except:
        avg_req_RR = np.full((10),None).tolist()



    # NUM OF BOUNDARIES
    num_6s = reduce_len(
        arr = make_len_constant(num_6s_overs,length=50),
        normalise=False,
        factor = 5,
        func='sum',
    )
    num_4s = reduce_len(
        arr=make_len_constant(num_4s_overs,length=50),
        normalise=False,
        factor = 5,
        func='sum'
    )
    num_wickets = reduce_len(
        arr=make_len_constant(num_wickets_overs,length=50),
        normalise=False,
        factor = 5,
        func='sum'
    )



    # DIST OF BOUNDARIES
    dist_num_6s = reduce_len(
        arr = make_len_constant(num_6s_overs,length=50),
        normalise=True,
        factor = 5,
        func='sum'
    )
    dist_num_4s = reduce_len(
        arr=make_len_constant(num_4s_overs,length=50),
        normalise=True,
        factor = 5,
        func='sum'
    )
    dist_num_wickets = reduce_len(
        arr=make_len_constant(num_wickets_overs,length=50),
        normalise=True,
        factor = 5,
        func='sum'
    )


    tmp = pd.DataFrame(
    [
        np.full((len(overs)), row['match_num']).tolist(),
        np.full((len(overs)), row['team']).tolist(),
        np.full((len(overs)), row['toss']).tolist(),
        np.full((len(overs)), row['toss_result']).tolist(),
        overs,
        avg_RR,
        avg_req_RR,
        num_6s,
        num_4s,
        num_wickets,
        dist_num_6s,
        dist_num_4s,
        dist_num_wickets,
        np.full((len(overs)), row['match_result']).tolist(),
    ]
    ).T.values.tolist()
    
#     print(tmp)
#     break
#     print(num)
#     num+=1

    vals.extend(tmp)
    
    row = cursor.fetchone()
    


insert(query,vals,commit=True)
    
    

Connected...
Executing...
Done...
Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

# dist_bowling_perf table

In [23]:
def make_len_constant(arr,length=50, replace_val=None, dtype=float):
    return np.pad(np.array(arr, dtype=dtype), (0, length-len(arr)),'constant', constant_values=replace_val)


def reduce_len(arr, factor,normalise=False,func='mean', nan_replace_val=None):
    
    if func == 'mean':
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).mean(axis=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).mean(axis=1).replace(np.nan,None).to_list()
 
    if func == 'median':
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).median(axis=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).median(axis=1).replace(np.nan,None).to_list()

    if func == 'sum':
        #         pandas fix issue of if all na, it returns 0.0, to fix do min_count=1
        if normalise:
            tmp = pd.DataFrame(arr.reshape(-1,factor)).sum(axis=1,min_count=1)

            return (tmp/max(tmp)).replace(np.nan,nan_replace_val).to_list() 
        else:    
            return pd.DataFrame(arr.reshape(-1,factor)).sum(axis=1,min_count=1).replace(np.nan,None).to_list()
    
    
    


table_name = "dist_bowling_perf"

q = f"""
drop table if exists {table_name};
"""

execute(q)


q = f"""
create table {table_name} (
match_num int NOT NULL,
team varchar(30) not null,

toss varchar(20),
toss_result varchar(20),

overs varchar(8),

num_wickets_taken int,
dist_wickets_taken DECIMAL(6,4),

num_extras_given int,
dist_extras_given DECIMAL(6,4),

match_result varchar(20) not null
);
"""

execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

In [24]:
dbconfig = read_db_config()
conn = MySQLConnection(**dbconfig)

cursor = conn.cursor(dictionary=True)

cols = "(match_num, team, toss, toss_result, overs, num_wickets_taken, dist_wickets_taken, num_extras_given, dist_extras_given, match_result)"
tmp = "values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) "
vals = []

query = f"insert into {table_name} {cols} "+tmp


q = """

SELECT 
b.match_num,
b.team,
b.toss,
b.toss_result,

CASE WHEN b.toss_result = "BAT" THEN o.data -> "$.inningOvers[1].stats[*]" WHEN b.toss_result = "BOWL" THEN o.data ->> "$.inningOvers[0].stats[*]" END AS overs_data,

b.match_result

FROM scoreboard b
LEFT JOIN overs o ON b.match_num = o.match_num
ORDER BY b.match_num;

"""


cursor.execute(q)

row = cursor.fetchone()

vals = []
num = 1
while row is not None:
    
    num_wickets_taken_overs = []
    num_extras_given_overs = []

    tmp_over_wickets_taken = []
    tmp_overs_extras_given = []

    try:
        for i in json.loads(row['overs_data']):
            for j in i['balls']:
                tmp_over_wickets_taken.append(j['isWicket'])
                tmp_overs_extras_given.extend( [j['byes'], j['wides'], j['legbyes'], j['noballs']])

            num_wickets_taken_overs.append(np.sum(np.array(tmp_over_wickets_taken,dtype=int)))
            num_extras_given_overs.append(np.sum(np.array(tmp_overs_extras_given, dtype=int)))

            tmp_over_wickets_taken = []
            tmp_overs_extras_given = []

    except:
        num_wickets_taken_overs = np.full((50),None).tolist()
        num_extras_given_overs = np.full((50),None).tolist()


    overs = ['0-5','5-10','10-15','15-20','20-25','25-30','30-35','35-40','40-45','45-50']
    
        # NUM OF WICKETS
    num_wickets_taken = reduce_len(
        arr = make_len_constant(num_wickets_taken_overs,length=50),
        normalise=False,
        factor = 5,
        func='sum',
    )
        # NUM OF EXTRAS
    num_extras_given = reduce_len(
        arr=make_len_constant(num_extras_given_overs,length=50),
        normalise=False,
        factor = 5,
        func='sum'
    )



    # DIST OF WICKETS
    dist_wickets_taken = reduce_len(
        arr = make_len_constant(num_wickets_taken_overs,length=50),
        normalise=True,
        factor = 5,
        func='sum'
    )
    # DIST OF EXTRAS
    dist_extras_given = reduce_len(
        arr=make_len_constant(num_extras_given_overs,length=50),
        normalise=True,
        factor = 5,
        func='sum'
    )
    
    tmp = pd.DataFrame(
    [
        np.full((len(overs)), row['match_num']).tolist(),
        np.full((len(overs)), row['team']).tolist(),
        np.full((len(overs)), row['toss']).tolist(),
        np.full((len(overs)), row['toss_result']).tolist(),
        overs,
        num_wickets_taken,
        dist_wickets_taken,
        num_extras_given,
        dist_extras_given,
        np.full((len(overs)), row['match_result']).tolist(),
    ]
    ).T.values.tolist()
    
#     print(tmp)


#     print(num)
    num+=1

    vals.extend(tmp)
    
    row = cursor.fetchone()
    


insert(query,vals,commit=True)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
Connected...
Executing...
Done...


[]

# Create procedures and tables

## all teams with team_id 

In [25]:
q = """
drop procedure if exists refresh_teams;
"""
execute(q)

q = """


CREATE PROCEDURE refresh_teams()
BEGIN
    drop table if exists teams;
    create table teams as 
    WITH cte AS(
        SELECT team1 AS team, id_team1 AS team_id FROM main
        UNION
        SELECT team2 AS team, id_team2 AS team_id FROM main
    )

    SELECT DISTINCT 
    *
    FROM cte
    ORDER BY team;
END

"""

execute(q)

q = """call refresh_teams();"""
execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

## points table

In [26]:
q = """
drop procedure if exists refresh_points_table;
"""
execute(q)



# Points table procedure
q = """
CREATE PROCEDURE refresh_points_table()
BEGIN
    
    drop table if exists points_table;
    
    create table points_table as

    WITH with_position AS (
    WITH with_total_pts AS (
    WITH with_grp AS(
    WITH points AS (
    WITH cte2 AS (
    WITH cte1 AS (
    SELECT match_num,team1,team2, 
    CASE WHEN team1=winner THEN 2 WHEN winner = 'abandoned' OR winner = 'tied' THEN 1 ELSE 0 END AS points_team1,
    CASE WHEN team2=winner THEN 2 WHEN winner = 'abandoned' OR winner = 'tied' THEN 1 ELSE 0 END AS points_team2
    FROM main
    WHERE match_num <=45
    )

    SELECT 
    match_num, team1 AS team, points_team1 AS pts
    FROM cte1

    UNION

    SELECT 
    match_num, team2 AS team, points_team2 AS pts
    FROM cte1
    )
    SELECT
    *,
    sum(pts) over(PARTITION BY team ORDER BY match_num ROWS BETWEEN unbounded preceding AND current ROW) AS total_pts
    FROM cte2
    )

    SELECT m.match_num, t.team, p.total_pts, count(p.total_pts) over(PARTITION BY team ORDER BY match_num) AS _grp
    FROM teams t
    CROSS JOIN main m
    LEFT JOIN points p ON m.match_num = p.match_num AND t.team = p.team
    )

    SELECT 
    match_num,
    team,
    first_value(total_pts) over(PARTITION BY team,_grp ORDER BY _grp ) AS total_pts
    FROM with_grp

    )

    SELECT
    *,
    rank() over(PARTITION BY match_num ORDER BY total_pts DESC) AS position
    FROM with_total_pts
    )

    SELECT match_num,team,total_pts, CASE WHEN total_pts IS NULL THEN NULL ELSE position END AS position
    FROM with_position;
END

"""

execute(q)

q = """call refresh_points_table();"""
execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

## Scoreaboard

In [ ]:
q = """
drop procedure if exists refresh_scoreboard;
"""
execute(q)





q = """
create procedure refresh_scoreboard()
BEGIN
    drop table if exists scoreboard;

    create table scoreboard as
        WITH with_toss_result AS(
          WITH with_winner_id AS(
              -- for team1
              SELECT
              -- match num
              s.match_num,

              -- team
              m.team1 AS team,

              cast(CASE WHEN `data` ->> "$.match.tossWinnerTeamId"='null' THEN NULL ELSE `data` ->> "$.match.tossWinnerTeamId" END AS UNSIGNED) AS toss_winner_id,

              CASE WHEN `data` ->> "$.content.innings[0].team.name" = m.team1 THEN "BAT" ELSE "BOWL" END AS toss_result


              FROM scorecard s
              LEFT JOIN main m ON s.match_num = m.match_num

            UNION

              -- for team 2
              SELECT
              -- match num
              s.match_num,

              -- team
              m.team2 AS team,

              cast(CASE WHEN `data` ->> "$.match.tossWinnerTeamId"='null' THEN NULL ELSE `data` ->> "$.match.tossWinnerTeamId" END AS UNSIGNED) AS toss_winner_id,

              CASE WHEN `data` ->> "$.content.innings[0].team.name" = m.team2 THEN "BAT" ELSE "BOWL" END AS toss_result


              FROM scorecard s
              LEFT JOIN main m ON s.match_num = m.match_num
          )

          SELECT
          a.match_num,
          a.team,
          CASE WHEN a.team = b.team THEN "WON" WHEN a.team <> b.team THEN "LOST" END AS toss,
          a.toss_result


          FROM with_winner_id a
          LEFT JOIN teams b ON a.toss_winner_id = b.team_id
          ORDER BY a.match_num
        )

        SELECT
        c.*,

        -- score
        CASE WHEN c.toss_result = "BAT" THEN s.data ->> "$.content.innings[0].runs" WHEN c.toss_result = "BOWL" THEN s.data ->> "$.content.innings[1].runs" END AS score_runs,

        -- wickets
        CASE WHEN c.toss_result = "BAT" THEN s.data ->> "$.content.innings[0].wickets" WHEN c.toss_result = "BOWL" THEN s.data ->> "$.content.innings[1].wickets" END AS score_wickets,

        -- balls played
        CASE WHEN c.toss_result = "BAT" THEN s.data ->> "$.content.innings[0].balls" WHEN c.toss_result = "BOWL" THEN s.data ->> "$.content.innings[1].balls" END AS score_balls_faced,


        -- match result
        CASE WHEN c.team = m.winner THEN "WON" WHEN m.winner NOT IN (SELECT team FROM teams) THEN m.winner ELSE "LOST" END AS match_result

        FROM with_toss_result c
        LEFT JOIN scorecard s ON c.match_num = s.match_num
        LEFT JOIN main m ON c.match_num = m.match_num
        ORDER BY c.match_num;

END
"""

execute(q)

q = """call refresh_scoreboard();"""
execute(q)

## Bowling stats

In [ ]:
q = """
drop procedure if exists refresh_bowling_perf;
"""
execute(q)

q = """

create procedure refresh_bowling_perf()
begin
    drop table if exists bowling_perf;
    
    create table bowling_perf as
        SELECT 
        b.match_num,
        b.team,
        b.toss,
        b.toss_result,


        -- wickets taken
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].wickets" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].wickets" END AS wickets_taken,

        -- extras given
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].extras" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].extras" END AS extras_given,

        -- wides given
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].wides" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].wides" END AS wides_given,

        -- legbyes given
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].legbyes" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].legbyes" END AS legbyes_given,

        -- byes given
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].byes" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].byes" END AS byes_given,

        -- noballs given
        CASE WHEN b.toss_result = "BAT" THEN c.data ->> "$.content.innings[1].noballs" WHEN b.toss_result = "BOWL" THEN c.data -> "$.content.innings[0].noballs" END AS noballs_given,


        b.match_result

        FROM scoreboard b
        LEFT JOIN scorecard c ON b.match_num = c.match_num
        ORDER BY b.match_num;


end

"""

execute(q)

q = """call refresh_bowling_perf();"""
execute(q)

## ground wise 6s and 4s for parallel line graph

In [13]:
q = """
drop procedure if exists refresh_groundwise_boundaries_graph;
"""
execute(q)

q = """

create procedure refresh_groundwise_boundaries_graph()
BEGIN
    drop table if exists groundwise_boundaries_graph;
    
    create table groundwise_boundaries_graph as
        WITH bat_1st AS (
        SELECT 
            b.match_num,
            m.ground,
            (b.num_6s) AS bat_1st_6s, 
            (b.num_4s) AS bat_1st_4s
        FROM batting_perf b
        LEFT JOIN main m ON m.match_num = b.match_num
        WHERE b.toss_result = "BAT" AND m.margin IS NOT NULL 
        ),
        bat_2nd AS(
        SELECT 
            b.match_num,
            m.ground,
            (b.num_6s) AS bat_2nd_6s, 
            (b.num_4s) AS bat_2nd_4s
        FROM batting_perf b
        LEFT JOIN main m ON m.match_num = b.match_num
        WHERE b.toss_result = "BOWL" AND m.margin IS NOT NULL 
        )

        SELECT f.match_num, f.ground, f.bat_1st_6s, f.bat_1st_4s, s.bat_2nd_6s, s.bat_2nd_4s
        FROM bat_1st AS f
        LEFT JOIN bat_2nd s ON s.match_num = f.match_num;

END

"""




execute(q)

q = """call refresh_groundwise_boundaries_graph();"""
execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

## Unpivoted batting performance by batting order

In [3]:
q = """
drop procedure if exists refresh_unpivot_batting_order_batting_perf;
"""
execute(q)

q = """

create procedure refresh_unpivot_batting_order_batting_perf()
BEGIN
    drop table if exists unpivot_batting_order_batting_perf;
    
    create table unpivot_batting_order_batting_perf as 
    
        WITH bat_first AS (
            SELECT 
            match_num,
            team,
            toss AS bat_first_toss,
            overall_RR AS bat_first_overall_RR,
            avg_RR AS bat_first_avg_RR,
            num_6s AS bat_first_num_6s,
            num_4s AS bat_first_num_4s,
            match_result AS bat_first_match_result

            FROM batting_perf
            WHERE toss_result ="BAT" AND toss IS NOT NULL
            ),

        bat_second AS (
            SELECT
            match_num,
            team,
            toss AS bat_second_toss,
            overall_RR AS bat_second_overall_RR,
            avg_RR AS bat_second_avg_RR,
            num_6s AS bat_second_num_6s,
            num_4s AS bat_second_num_4s,
            match_result AS bat_second_match_result

            FROM batting_perf
            WHERE toss_result ="BOWL" AND toss IS NOT NULL
            )


        SELECT
        a.*,
        NULL as bat_second_toss,
        NULL as bat_second_overall_RR,
        NULL as bat_second_avg_RR,
        NULL as bat_second_num_6s,
        NULL as bat_second_num_4s,
        NULL as bat_second_match_result

        FROM bat_first a

        UNION

        SELECT
        match_num,
        team,
        NULL,NULL,NULL,NULL,NULL, NULL,
        bat_second_toss,
        bat_second_overall_RR,
        bat_second_avg_RR,
        bat_second_num_6s,
        bat_second_num_4s,
        bat_second_match_result

        FROM bat_second;


END

"""

execute(q)

q = """call refresh_unpivot_batting_order_batting_perf();"""

execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]

## unpivoted batting performance by toss

In [4]:
q = """
drop procedure if exists refresh_unpivot_toss_batting_perf;
"""
execute(q)

q = """

create procedure refresh_unpivot_toss_batting_perf()
BEGIN
    drop table if exists unpivot_toss_batting_perf;
    
    create table unpivot_toss_batting_perf as 
    
    WITH won_toss AS (
        SELECT 
        match_num,
        team,
        toss_result AS won_toss_toss_result,
        overall_RR AS won_toss_overall_RR,
        avg_RR AS won_toss_avg_RR,
        num_6s AS won_toss_num_6s,
        num_4s AS won_toss_num_4s,
        match_result AS won_toss_match_result

        FROM batting_perf
        WHERE toss ="WON" 
        ),

    lost_toss AS (
        SELECT
        match_num,
        team,
        toss_result AS lost_toss_toss_result,
        overall_RR AS lost_toss_overall_RR,
        avg_RR AS lost_toss_avg_RR,
        num_6s AS lost_toss_num_6s,
        num_4s AS lost_toss_num_4s,
        match_result AS lost_toss_match_result

        FROM batting_perf
        WHERE toss ="LOST" 

        )

    SELECT
    a.*,
    NULL AS lost_toss_toss_result,
    NULL AS lost_toss_overall_RR,
    NULL AS lost_toss_avg_RR,
    NULL AS lost_toss_num_6s,
    NULL AS lost_toss_num_4s,
    NULL AS lost_toss_match_result
    FROM won_toss a

    UNION

    SELECT
    match_num,
    team,
    NULL,NULL,NULL,NULL,NULL, NULL,
    lost_toss_toss_result,
    lost_toss_overall_RR,
    lost_toss_avg_RR,
    lost_toss_num_6s,
    lost_toss_num_4s,
    lost_toss_match_result

    FROM lost_toss;


END

"""

execute(q)

q = """call refresh_unpivot_toss_batting_perf();"""

execute(q)

Connected...
Executing...
Done...
Connected...
Executing...
Done...
Connected...
Executing...
Done...


[]